In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip -q -o /kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
!unzip -q -o /kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip

In [ ]:
import glob
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *

In [ ]:
glob.glob('/kaggle/working/train/*')[:5]

In [ ]:
Image.open('/kaggle/working/train/dog.11795.jpg')

In [ ]:
Image.open('/kaggle/working/train/cat.8986.jpg')

In [ ]:
train = pd.DataFrame({'path':glob.glob('/kaggle/working/train/*')})
train['label'] = train['path'].apply(lambda x: x.split('.')[0].split('/')[-1])
train

In [ ]:
# valid 데이터셋
from sklearn.model_selection import train_test_split
X_train,X_valid = train_test_split(train,test_size=0.2,random_state=42,stratify=train['label'])

In [ ]:
idg = ImageDataGenerator()
train_generator = idg.flow_from_dataframe(X_train,x_col='path',y_col='label',target_size=(200,200),batch_size=100)
valid_generator = idg.flow_from_dataframe(X_valid,x_col='path',y_col='label',target_size=(200,200),batch_size=100)

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
efn = EfficientNetB0(include_top=False,pooling='avg')
efn.summary()

In [ ]:
model=Sequential()
model.add(efn)
model.add(Dense(2,activation='softmax'))
model.summary()
model.compile(metrics=['acc'],loss='categorical_crossentropy',optimizer='adam')
model.fit(train_generator,validation_data=valid_generator,epochs=3)

In [ ]:
test = pd.DataFrame({'path':glob.glob('/kaggle/working/test/*')})
test

In [ ]:
test_generator = idg.flow_from_dataframe(test,x_col='path',y_col=None,target_size=(200,200),batch_size=100,
                                        class_mode=None,shuffle=False)
result = model.predict(test_generator,verbose=1,workers=3)
result

In [ ]:
sub = pd.read_csv('/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')
sub['id'] = test['path'].apply(lambda x: x.split('/')[-1].split('.')[0])
sub['label'] = result[:,1]
sub.to_csv('cd_result.csv',index=False)